In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../")

In [2]:
from xent import M, device
from xent.datasets import Wikipedia
from xent.tasks import Closure

from datasets import load_dataset

In [3]:
model = M("gpt2", "M0")
task = Closure(model)

In [4]:
data = load_dataset(
    "wikipedia", 
    "20220301.en", 
    trust_remote_code=True,
    )["train"]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
data_1 = load_dataset(
    "wikipedia", 
    "20220301.en", 
    trust_remote_code=True,
    split="train[:50%]"
    )

data_2 = load_dataset(
    "wikipedia", 
    "20220301.en", 
    trust_remote_code=True,
    split="train[50%:]"
    )

data_slices = load_dataset(
    "wikipedia", 
    "20220301.en", 
    trust_remote_code=True,
    split=[f"train[{k}%:{k+10}%]" for k in range(0, 100, 10)]
)


In [ ]:
data_1 = data.select(range(0,len(data)//2))
data_2 = data.select(range(len(data)//2,0))

In [ ]:
n_shards = 7
shards = []
for s in range(n_shards):
    shards.append(data_1.shard(num_shards=n_shards, index=s))


In [ ]:
import torch
generated = torch.LongTensor([]).to(device)
for n in range(1):
    get_data = lambda: data[n]["text"]
    output = task.generate_parallel_parallel(get_data)

slice shape: torch.Size([1, 202])
task shape: torch.Size([1, 19])
slice again: torch.Size([1, 202])
task again: torch.Size([1, 19])


../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [43,0,0], thread: [0,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [43,0,0], thread: [1,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(model.detokenize(output[0][0]))
print(model.detokenize(output[4][0]))

Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.

Humans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in 

In [ ]:
generated.shape

torch.Size([0])

In [ ]:
generated.shape

torch.Size([0])

In [ ]:
import torch

t = torch.ones(10)
t = t.unfold(0,3,3)

for row in t:
    print(row)


tensor([1., 1., 1.])
tensor([1., 1., 1.])
tensor([1., 1., 1.])


In [ ]:
get_corpus_method = lambda: data.database[91]["text"]

In [ ]:
new_data = task.synthesize(get_corpus_method, 1, out_type="tokens", inverse_order=True)

TypeError: Closure.generate() got an unexpected keyword argument 'inverse_order'

In [ ]:
total = 153
chunks = 30

def schedule(total, chunks):
    if total%chunks != 0:
        return [*[chunks]*(total//chunks), total%chunks]
    else: 
        return [chunks]*(total//chunks)

schedule(total, chunks)

[30, 30, 30, 30, 30, 3]

In [ ]:
workers = 3

def distribute_workers(total, chunks, workers):
    splits = schedule(total, workers)
    return splits
    schedules = [schedule(s, chunks) for s in splits]
    return schedules

total = 100
workers = 3
chunks = 100

per_worker = total // workers
per_worker_schedule = [[*[chunks]*(per_worker//chunks), total%chunks] for _ in range(workers)]

import torch
torch.tensor(per_worker_schedule).sum()

tensor(0)

In [ ]:
def split_into_n(total, n):
    base = total // n
    remainder = total % n
    result = [base] * n
    if remainder:
        result[-1] += remainder
    return result

split_into_n(6458670, 3)  # Returns [33, 33, 34]

[2152890, 2152890, 2152890]